In [2]:
import telebot
import requests
from telebot import types
from pymongo import MongoClient

In [1]:
!python --version

Python 3.8.5


# Create Database

In [2]:
myclient = MongoClient("mongodb://localhost:27017/")
mydb = myclient["sentence_db"]
sent_col = mydb["sentence_table"]
valid_col = mydb["validation_table"]
myclient.list_database_names()

# Check Connection

In [3]:
dblist = myclient.list_database_names()
if "sentence_db" in dblist:
  print("The database exists.")

# Adding data to MongoDB

In [ ]:
mylist = []
f = open('dev.fa', 'r', encoding="utf8")
for i, line in enumerate(f.readlines()):
    print(i)
    tmpdict = {}
    tmpdict['_id'] = i
    tmpdict['txt'] = str(line.strip('\n'))
    tmpdict['choose_num'] = 0
    mylist.append(tmpdict)

x = sent_col.insert_many(mylist)


# Random Sampling

In [ ]:
def random_sample():
    a = sent_col.aggregate([{"$sample": {"size": 1}}, {"$match": {"choose_num": {"$lte": 5}}}])
    sample = list(a)[0]
#     choose_num = sample['choose_num']
    return sample

def random_validation():
    a = valid_col.aggregate([{"$sample": {"size": 1}}, {"$match": {"valid_cnt": {"$lte": 5}}}])
    sample = list(a)[0]
#     choose_num = sample['choose_num']
    return sample

def update_choose_num(id, ch_n):
    myquery = {"_id": id}
    newvalues = {"$set": {"choose_num": ch_n+1}}
    sent_col.update_one(myquery, newvalues)
    
def update_valid_cnt(id, v_cnt):
    myquery = {"_id": id}
    newvalues = {"$set": {"valid_cnt": v_cnt+1}}
    valid_col.update_one(myquery, newvalues)

def update_validation(id, validation):
    myquery = {"_id": id}
    newvalues = {"$push": {"validations": validation}}
    valid_col.update_one(myquery, newvalues)

rs = random_validation()
print(rs)

# Recording Bot

In [ ]:
TOKEN="1402079733:AAH3rvaEdhYehfyTTwie5JHVKnnYSsi9RwI"
bot = telebot.TeleBot(TOKEN, parse_mode=None)
last_id = []
last_choose_num = []
user = bot.get_me()
chat_id = user.id
updates = bot.get_updates()
data_entries = []

class Entry:
    def __init__(self, text, audio_path, is_validated):
        self.text = text
        self.audio_path = audio_path
        self.is_validated = is_validated


cmd1 = types.BotCommand('new', 'record new sentence')

@bot.message_handler(commands=['start'])
def new_sentence(message):
    bot.send_message(message.from_user.id, "سلام. به بات من خوش آمدید.")
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
    speak_button = types.KeyboardButton(text="جمله ی جدید")
    keyboard.add(speak_button)
    bot.send_message(message.chat.id, "برای دریافت جمله ی جدید، دکمه ی زیر رو فشار بدین", reply_markup=keyboard)

    
# @bot.message_handler(commands=['start', 'help'])
# def send_welcome(message):
#     bot.reply_to(message, "سلاام، به بات من خوش آمدید. برای شروع لطفا دستور '/new' را وارد کنید.")


@bot.message_handler(content_types=['text'])
def handle_message(message):
    if message.text == "جمله ی جدید":
        bot.send_message(message.from_user.id, "جمله ی زیر رو بخونید: ")
        sample = random_sample()
        print(sample)
        bot.send_message(message.from_user.id, sample['txt'])
        last_id.append(sample['_id'])
        last_choose_num.append(sample['choose_num'])


@bot.message_handler(content_types=['voice'])
def handle_voice(message):
    file_info = bot.get_file(message.voice.file_id)
    chat_id = message.from_user.id
    try: 
        file = requests.get('https://api.telegram.org/file/bot{0}/{1}'.format(TOKEN, file_info.file_path))
        filepath = "voice/{}_{}.wav".format(str(last_id[-1]), str(last_choose_num[-1]))
        with open(filepath, mode='bx') as f:
            f.write(file.content)
        f.close()
        bot.send_message(chat_id, "صدای شما ضبط شد.")
        update_choose_num(last_id[-1], last_choose_num[-1])
        text = sent_col.find_one({"_id": last_id[-1]})['txt']
        col_record = valid_col.insert_one({"_id": "{}_{}".format(str(last_id[-1]), str(last_choose_num[-1])), "txt": text, "filepath": filepath, "valid_cnt": 0, "validations": []})
    except Exception as e:
        print(e)

bot.polling()

# Validation Bot

In [ ]:
TOKEN="1631882992:AAFLWcnyWZBaCBhDgCH1HpadBoAXLlAw210"
bot = telebot.TeleBot(TOKEN, parse_mode=None)
last_id = []
last_valid_cnt = []
user = bot.get_me()
chat_id = user.id
updates = bot.get_updates()
data_entries = []

class Entry:
    def __init__(self, text, audio_path, is_validated):
        self.text = text
        self.audio_path = audio_path
        self.is_validated = is_validated


cmd1 = types.BotCommand('new', 'record new sentence')


# @bot.message_handler(commands=['start', 'help'])
# def send_welcome(message):
#     bot.reply_to(message, "سلاام، به بات من خوش آمدید. برای شروع لطفا دستور '/new' را وارد کنید.")


@bot.message_handler(commands=['new'])
def new_sentence(message):
    bot.send_message(message.from_user.id, "سلام. به بات ارزیابی خوش آمدید.")
    keyboard = types.ReplyKeyboardMarkup(row_width=1, one_time_keyboard=True)
    valid_button = types.KeyboardButton(text="ارزیابی جدید")
    keyboard.add(valid_button)
    bot.send_message(message.chat.id, "برای دریافت جمله ی جدید، دکمه ی زیر رو فشار بدین", reply_markup=keyboard)



@bot.message_handler(content_types=['text'])
def handle_message(message):
    if message.text == "ارزیابی جدید":
        bot.send_message(message.from_user.id, "جمله و صدای زیر را از نظر مطابقت متن و صدا، و کیفیت صدا ارزیابی کنید: ")
        sample = random_validation()
        print(sample)
        bot.send_message(message.from_user.id, sample['txt'])
        voice = open(sample['filepath'], 'rb')
        bot.send_voice(message.from_user.id, voice)
        last_id.append(sample['_id'])
        last_valid_cnt.append(sample['valid_cnt'])
        keyboard2 = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
        good_btn = types.KeyboardButton(text="خوب")
        bad_btn = types.KeyboardButton(text="بد")
        keyboard2.add(good_btn, bad_btn)
        bot.send_message(message.chat.id, "ارزیابی خودتون را ثبت کنید", reply_markup=keyboard2)
    elif message.text == "خوب":
        update_valid_cnt(last_id[-1], last_valid_cnt[-1])
        update_validation(last_id[-1], 1)
        bot.send_message(message.chat.id, "ارزیابی شما: {}".format("خوب"))
        keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
        valid_button = types.KeyboardButton(text="ارزیابی جدید")
        keyboard.add(valid_button)
        bot.send_message(message.chat.id, "برای دریافت جمله ی جدید، دکمه ی زیر رو فشار بدین", reply_markup=keyboard)
        print("خوب")
    elif message.text == "بد":
        update_valid_cnt(last_id[-1], last_valid_cnt[-1])
        update_validation(last_id[-1], 0)
        bot.send_message(message.chat.id, "ارزیابی شما: {}".format("بد"))
        keyboard = types.ReplyKeyboardMarkup(row_width=1, one_time_keyboard=True)
        valid_button = types.KeyboardButton(text="ارزیابی جدید")
        keyboard.add(valid_button)
        bot.send_message(message.chat.id, "برای دریافت جمله ی جدید، دکمه ی زیر رو فشار بدین", reply_markup=keyboard)
        print("بد")
    
    
        
        
@bot.message_handler(content_types=['voice'])
def handle_voice(message):
    file_info = bot.get_file(message.voice.file_id)
    chat_id = message.from_user.id
    try: 
        file = requests.get('https://api.telegram.org/file/bot{0}/{1}'.format(TOKEN, file_info.file_path))
        with open("{}_{}.wav".format(str(last_id[-1]), str(last_choose_num[-1])), mode='bx') as f:
            f.write(file.content)
        bot.send_message(chat_id, "صدای شما ضبط شد.")
        update_choose_num(last_id[-1], last_choose_num[-1])
    except:
        print("error in response")

bot.polling()